In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost 

from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from sklearn.metrics import accuracy_score
from time import time
from google.colab import drive
from time import perf_counter
import xgboost as xgb
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Preprocess

In [ ]:
# Dataset with deletion of NaN
df_clean =  pd.read_csv('/gdrive/MyDrive/X_station_train_clean.csv', index_col=0)

# Dataset with imputation of NaN 
# df_imputation = pd.read_csv('/gdrive/MyDrive/X_station_Train_imputation.csv', index_col=0)

# 1rst Dataset with NaN
df_full_dataset = pd.read_csv('/gdrive/MyDrive/X_station_train.csv')

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
def preprocessing_X_station (X_station_path, stations_coordinates_path, L_labels):
  
  # Renaming of features and add stations coordinates

    # Open Data
    X_station = pd.read_csv(X_station_path)
    stations_coordinates = pd.read_csv(stations_coordinates_path)

    # Split Date
    if 'X_station_train' in X_station_path:
        X_station['year']  = X_station['date'].apply(lambda row: row[:4]).astype('int32')
        X_station['month'] = X_station['date'].apply(lambda row: row[5:7]).astype('int32')
        X_station['day']   = X_station['date'].apply(lambda row: row[8:10]).astype('int32')
        X_station['hour']  = X_station['date'].apply(lambda row: row[11:13]).astype('int32')
        X_station.drop("date", axis='columns', inplace=True)

    if 'X_station_test' in X_station_path:
        X_station['number_sta'] = X_station['Id'].apply(lambda row: row.split('_')[0]).astype('int32')
        X_station['day_id']     = X_station['Id'].apply(lambda row: row.split('_')[1]).astype('int32')
        X_station['hour']       = X_station['Id'].apply(lambda row: row.split('_')[2]).astype('int32')


    X_station.drop("Id", axis='columns', inplace=True)


    # Add Stations Coordinates
    X_station = pd.merge(X_station, stations_coordinates, how='left', on='number_sta')

    # Rename columns
    X_station.rename(columns={'number_sta':"station_id", 
                                    'ff': "wind_speed", 
                                    't': "temperature", 
                                    'td':"dew_point", 
                                    'hu':"humidity", 
                                    'dd':"wind_direction", 
                                    'precip':"precipitations",
                                    'lat':"latitude", 
                                    'lon':"longitude", 
                                    'height_sta':"altitude"}, inplace=True)

    # Reorder columns
    X_station = X_station[L_labels]

    # Sort by station and date
    if 'X_station_train' in X_station_path:
        X_station = X_station.sort_values(by=['station_id','year', 'month', 'day', 'hour'])
    if 'X_station_test' in X_station_path:
        X_station = X_station.sort_values(by=['station_id', 'day_id', 'hour'])

    X_station = X_station.reset_index(drop=True)

    return X_station

In [ ]:
def m_mape(y_true,y_predict):
    n = len(y_true)
    At = np.array(y_true) + 1
    Ft = np.array(y_predict) + 1

    res = ((100/n)*(np.sum(np.abs((Ft-At)/At))))
    return res

In [ ]:
# display(df_clean[df_clean['precipitations']>0])
X_station = df_clean.copy()
Y_station_train_regr = X_station.groupby(['station_id'])['precipitations'].shift(-1)
X_station.loc[X_station.precipitations>0,'precipitations']=1
print(X_station.shape)
Y_station_train = X_station.groupby(['station_id'])['precipitations'].shift(-1)


listNan=Y_station_train[Y_station_train.isna()].index.values.tolist()
if len(listNan)>0:
  X_station = X_station.drop(listNan)
  Y_station_train = Y_station_train.drop(listNan)
  Y_station_train_regr = Y_station_train_regr.drop(listNan)


X_train = X_station[X_station['year']==2016]
listX_train=X_station[X_station['year']==2016].index.values.tolist()
y_train=Y_station_train[listX_train]
y_train_regr = Y_station_train_regr[listX_train]

X_test = X_station[X_station['year']==2017]
listX_test=X_station[X_station['year']==2017].index.values.tolist()
y_test=Y_station_train[listX_test]
y_test_regr = Y_station_train_regr[listX_test]



(2264105, 15)


In [ ]:
def normalizing_data (X, L_labels, L_labels_cos_sin, min_train, max_train):

    # Exctact & Reorder columns
    X = X[L_labels]

    # Normalize
    X = (X - min_train) / (max_train - min_train)

    # Les valeurs sont normalisées entre 0 et 1, or cos(0)=cos(2*pi) => janvier=decembre, donc la plus grande valeur (normalisée) ne doit pas être 1
    X['month_cos'] = np.cos(2*np.pi * X['month'] * 11/12)  
    X['month_sin'] = np.sin(2*np.pi * X['month'] * 11/12)
    X['hour_cos'] = np.cos(2*np.pi * X['hour'] * 23/24)
    X['hour_sin'] = np.sin(2*np.pi * X['hour'] * 23/24)
    X['wind_direction_cos'] = np.cos(2*np.pi * X['wind_direction'] * 359/360)
    X['wind_direction_sin'] = np.sin(2*np.pi * X['wind_direction'] * 359/360)

    X.drop('month', axis=1, inplace=True)
    X.drop('hour', axis=1, inplace=True)
    X.drop('wind_direction', axis=1, inplace=True)

    # Reorder columns
    X = X[L_labels_cos_sin]

    # X_station = X_station.reset_index(drop=True)

    return X

# Deal with Imbalanced database

1) Resampling Your Dataset

*   Remove
*   Add

2 ) Generate Synthetic Samples

*  SMOTE or the Synthetic Minority Over-sampling Technique.


# Resampling Dataset : remove 

In [ ]:
# display(X_station['precipitations'].describe())
# display(X_station[X_station['precipitations']>0])
# count_class_0, count_class_1 = X_station.precipitations.value_counts()



def imalanced_remove_data(X_train,y_train):
  df_class_0 = X_train[X_train['precipitations'] == 0]

  df_class_1 = X_train[X_train['precipitations'] == 1]
  display(len(df_class_0), len(df_class_1))
    # plt.bar([0,1], height=[len(df_class_0),len(df_class_1)])
    # plt.xticks([0,1], [0,1]);
    # plt.show()

  df_class_0_under = df_class_0.sample(len(df_class_1))
    # print(df_class_0_under)

  X_train_class = pd.concat([df_class_0_under, df_class_1], axis=0)

  y_train_class = y_train[X_train_class.index]

  print('Random under-sampling:')
  # print(X_train_class.precipitations.value_counts())

  # X_train_class.precipitations.value_counts().plot(kind='bar', title='Count (target)')
  # print(X_train_class)
  return X_train_class,y_train_class

In [ ]:
X_train_balanced,y_train_balanced = imalanced_remove_data(X_train,y_train)
print(X_train_balanced.shape,y_train_balanced.shape)

check_for_nan = X_train_balanced.isnull().sum().sum()
print("Xtrain nan :",check_for_nan)
check_for_nan = y_train_balanced.isnull().sum().sum()
print("ytrain nan :",check_for_nan)

display(X_train_balanced["precipitations"].unique())

 #Normalize
L_labels = ['latitude', 'longitude', 'altitude', 'month', 'hour', 'wind_direction', 'wind_speed', 'temperature', 'humidity', 'dew_point', 'precipitations']
L_labels_cos_sin = ['latitude', 'longitude', 'altitude', 'month_cos', 'month_sin', 'hour_cos', 'hour_sin', 'wind_direction_cos', 'wind_direction_sin', 'wind_speed', 'temperature', 'humidity', 'dew_point', 'precipitations']

min_train = X_train[L_labels].min()
max_train = X_train[L_labels].max()

X_train_balanced = normalizing_data(X_train_balanced, L_labels, L_labels_cos_sin, min_train, max_train)
X_test_norm = X_test
X_test_norm = normalizing_data(X_test, L_labels, L_labels_cos_sin, min_train, max_train)
# display(X_train_balanced)

992589

115630

Random under-sampling:
(231260, 15) (231260,)
Xtrain nan : 0
ytrain nan : 0


array([0., 1.])

In [ ]:

# param_grid = {
#     "max_depth": [3, 4, 5, 7],
#     "learning_rate": [0.1, 0.01, 0.05],
#     "gamma": [0, 0.25, 1],
#     "reg_lambda": [0, 1, 10],
#     "scale_pos_weight": [1, 3, 5],
#     "subsample": [0.8],
#     "colsample_bytree": [0.5],
# }

# from sklearn.model_selection import GridSearchCV

# # Init classifier
# xgb_cl = xgb.XGBClassifier()

# # Init Grid Search
# grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=5, scoring="roc_auc")


# # Fit
# _ = grid_cv.fit(X_train_balanced, y_train_balanced)
# grid_cv.best_params_

KeyboardInterrupt: ignored

In [ ]:
my_classifier =xgb.XGBClassifier(
    max_depth=5,
    learning_rate=0.01,
    gamma=0.25,
    reg_lambda=1,
)

t1_start = perf_counter()              
my_classifier.fit(X_train_balanced, y_train_balanced)
t1_stop = perf_counter()
print("Time:", t1_stop-t1_start)

Time: 22.058795193999686


In [ ]:
X_test_norm.reset_index(drop=True, inplace=True)
# display(X_test_norm)
t1_start = perf_counter() 
y_predict = my_classifier.predict(X_test_norm)
t1_stop = perf_counter()
print("Time:", t1_stop-t1_start)
print("My MAPE =", m_mape(y_test,y_predict))
print("MSE =",mean_squared_error(y_test,y_predict))
save = y_predict

accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Time: 4.3649105679996865
My MAPE = 5.4534906458685555
MSE = 0.08787224874669694
Accuracy: 91.21%


In [ ]:
y_pluie = y_test_regr[y_predict == 1]

id = np.where(y_predict == 1)[0].tolist()
print(id)
X_pluie = X_test_norm.loc[id]

print(X_pluie.shape,y_pluie.shape)
X_train_pluie, X_test_pluie, y_train_pluie, y_test_pluie = train_test_split(X_pluie, y_pluie, test_size=0.33, random_state=42)

regr = xgb.XGBRegressor(n_estimators=20, 
                     max_depth=18,
                     gamma=3.430739184133814, 
                     min_child_weight = 8,
                     reg_alpha=180,
                     reg_lambda=0.7436396623675846,
                     random_state=123)

t1_start = perf_counter()              
regr.fit(X_train_pluie, y_train_pluie)
t1_stop = perf_counter()
print("Time:", t1_stop-t1_start)



[7, 151, 153, 162, 164, 165, 167, 168, 173, 174, 175, 176, 178, 182, 207, 208, 209, 210, 212, 237, 270, 272, 277, 278, 279, 280, 340, 343, 344, 345, 347, 348, 349, 350, 351, 352, 353, 354, 361, 369, 374, 666, 685, 686, 687, 688, 689, 708, 710, 711, 718, 719, 720, 732, 734, 735, 737, 738, 749, 750, 752, 755, 756, 775, 776, 777, 784, 807, 808, 819, 820, 821, 822, 823, 824, 825, 835, 836, 846, 847, 848, 849, 850, 854, 855, 856, 857, 885, 886, 887, 888, 889, 909, 921, 1091, 1264, 1373, 1374, 1375, 1376, 1378, 1387, 1400, 1424, 1425, 1426, 1427, 1467, 1468, 1469, 1490, 1518, 1519, 1520, 1541, 1542, 1543, 1544, 1546, 1547, 1583, 1584, 1585, 1596, 1603, 1614, 1631, 1632, 1633, 1635, 1663, 1689, 1691, 1692, 1693, 1694, 1696, 1734, 1775, 1781, 1838, 1889, 1890, 1924, 1925, 1929, 1930, 1932, 1938, 1939, 1940, 1949, 1950, 1951, 1952, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 2077, 2140, 2173, 2174, 2175, 2191, 2450, 2507, 2735, 2807, 2865, 2866, 2868, 2869, 2870, 2880, 2881, 288

In [ ]:
t1_start = perf_counter() 
y_predict_regressor = regr.predict(X_test_pluie)
t1_stop = perf_counter()

print(y_test_pluie.shape,y_predict_regressor.shape)
print("Time:", t1_stop-t1_start)

print("\nMAPE données juste pluie")
print("My MAPE =", m_mape(y_test_pluie,y_predict_regressor))
print("MSE =",mean_squared_error(y_test_pluie,y_predict_regressor))


for i,j in zip(id,y_predict_regressor) :
  y_predict[i]=j

print("\nMAPE toutes données")
print("My MAPE =", m_mape(y_test_regr,y_predict))
print("MSE =",mean_squared_error(y_test_regr,y_predict))

(21282,) (21282,)
Time: 0.03510145299969736

MAPE données juste pluie
My MAPE = 39.09448333068498
MSE = 1.2442115899833537

MAPE toutes données
My MAPE = 5.102892697277603
MSE = 0.18847708241439076


In [ ]:
display(y_test_regr[y_test_regr>0])
display(y_predict[y_predict>0])

8790       0.2
8807       0.2
8810       0.2
8811       0.8
8812       0.2
          ... 
4409451    1.6
4409452    0.6
4409453    0.2
4409463    0.2
4409464    0.2
Name: precipitations, Length: 117048, dtype: float64

array([0.38271013, 0.7077595 , 0.50525743, ..., 1.        , 1.        ,
       1.        ])

# Data test


In [ ]:
X_station_test_path = '/gdrive/MyDrive/X_station_test.csv'
stations_coordinates_path = '/gdrive/MyDrive/stations_coordinates.csv'
L_labels_test  = ['station_id', 'day_id', 'latitude', 'longitude', 'altitude',         'month',        'hour', 'wind_direction', 'wind_speed', 'temperature', 'humidity', 'dew_point', 'precipitations']
X_station_test = preprocessing_X_station(X_station_test_path, stations_coordinates_path, L_labels_test)
X_test = normalizing_data (X_station_test, L_labels, L_labels_cos_sin, min_train, max_train)
display(X_test)

,latitude,longitude,altitude,month_cos,month_sin,hour_cos,hour_sin,wind_direction_cos,wind_direction_sin,wind_speed,temperature,humidity,dew_point,precipitations
0,0.637166,0.616366,0.190789,0.866025,-0.500000,1.000000,0.000000,NaN,NaN,NaN,0.297266,NaN,NaN,0.0
1,0.637166,0.616366,0.190789,0.866025,-0.500000,0.965926,0.258819,NaN,NaN,NaN,0.287220,NaN,NaN,0.0
2,0.637166,0.616366,0.190789,0.866025,-0.500000,0.866025,0.500000,NaN,NaN,NaN,0.277339,NaN,NaN,0.0
3,0.637166,0.616366,0.190789,0.866025,-0.500000,0.707107,0.707107,NaN,NaN,NaN,0.264328,NaN,NaN,0.0
4,0.637166,0.616366,0.190789,0.866025,-0.500000,0.500000,0.866025,NaN,NaN,NaN,0.261858,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2304797,0.601206,0.977025,0.407895,0.500000,0.866025,0.258819,-0.965926,-0.999052,0.043523,0.119697,0.211133,0.747475,0.496482,0.0
2304798,0.601206,0.977025,0.407895,0.500000,0.866025,0.500000,-0.866025,-0.942949,0.332938,0.073989,0.200264,0.784848,0.496384,0.0
2304799,0.601206,0.977025,0.407895,0.500000,0.866025,0.707107,-0.707107,-0.996936,-0.078217,0.081552,0.194335,0.809091,0.497068,0.0
2304800,0.601206,0.977025,0.407895,0.500000,0.866025,0.866025,-0.500000,-0.304033,0.952661,0.066097,0.181653,0.825253,0.492377,0.0


In [ ]:
Y_pred_test1 = my_classifier.predict(X_test)
id = np.where(Y_pred_test1 == 1)[0].tolist()
X_pluie = X_test.loc[id]
Y_pred_test2 = regr.predict(X_pluie)

for i,j in zip(id,Y_pred_test2) :
  Y_pred_test1[i]=j


Y_pred_test1 = Y_pred_test1 * (max_train['precipitations'] - min_train['precipitations']) + min_train['precipitations']
pred_merged = pd.concat([X_station_test, pd.DataFrame(Y_pred_test1, columns=['Y_pred'])], axis=1)
pred_merged = pred_merged[['station_id',	'day_id', 'Y_pred']]
pred_merged['station_day_id'] = pred_merged[['station_id', 'day_id']].astype(str).apply(lambda x: '_'.join(x), axis=1)

pred_merged.drop(['station_id',	'day_id'], axis=1, inplace=True)
pred_merged = pred_merged[['station_day_id', 'Y_pred']]

pred_merged = pred_merged.groupby('station_day_id').agg(np.sum).reset_index()

baseline_obs = pd.read_csv('/gdrive/MyDrive/Baseline_observation_test.csv')
baseline_obs.drop('Prediction', axis=1, inplace=True)

baseline_obs = baseline_obs.rename(columns={"Id": "station_day_id"})
pred_merged = pd.merge(baseline_obs, pred_merged, how='inner', on=['station_day_id'])

pred_merged = pred_merged.rename(columns={'station_day_id': "Id", 'Y_pred':"Prediction"})

display(pred_merged)

,Id,Prediction
0,14066001_149,0.000000
1,14126001_149,0.405564
2,14137001_149,0.000000
3,14216001_149,0.000000
4,14296001_149,0.000000
...,...,...
85135,86137003_293,0.000000
85136,86165005_293,0.000000
85137,86273001_293,0.000000
85138,91200002_293,0.000000


In [ ]:
pred_merged.to_csv('/gdrive/MyDrive/MLDM-Prediction/xgboostClassifer_XgboostRegressor_data_normalisee2.csv',index=False)

In [ ]:
print(len(id))

46802


In [ ]:
print(pred_merged['Prediction'].describe())
display(len(Y_pred_test1[Y_pred_test1>0]))
display(len(Y_pred_test1[Y_pred_test1==0]))

count    85140.000000
mean         0.319894
std          1.011324
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         14.896975
Name: Prediction, dtype: float64


46802

2258000

In [ ]:
print(pred_merged['Prediction'].describe())
display(len(Y_pred_test1[Y_pred_test1>0]))
display(len(Y_pred_test1[Y_pred_test1==0]))

count    85140.000000
mean         0.507104
std          1.507873
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         17.925201
Name: Prediction, dtype: float64


46802

2258000